Initialisations to change the base directory of the entry scripts and to update mxnet to the newest mxnet-mkl

In [ ]:
!sed -i 's/examples.//g' battlesnake_src/train.py
!sed -i 's/examples.//g' battlesnake_src/networks/agent.py
!sed -i '1i import subprocess\nsubprocess.run(["pip",  "uninstall", "mxnet-mkl", "-y"])\nsubprocess.run(["pip",  "install", "mxnet-mkl", "--pre"])\nsubprocess.run(["pip",  "install", "pandas"])' battlesnake_src/train.py

In [ ]:
import boto3
import sagemaker
import numpy as np

from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.mxnet.estimator import MXNet

In [ ]:
run_hpo = False
map_size = (15, 15)

## Initialise sagemaker
We need to define several parameters prior to running the training job. 
note: `local_mode` defines whether to run the code within this notebook or to run a sagemaker training job

In [ ]:
sage_session = sagemaker.session.Session()
s3_bucket = sage_session.default_bucket()  
s3_output_path = 's3://{}/'.format(s3_bucket)
print("S3 bucket path: {}".format(s3_output_path))

# Run on local mode if no HPO is required
if run_hpo == False:
    local_mode = True
else:
    local_mode = False
    
if local_mode:
    train_instance_type = 'local'
else:
    train_instance_type = "ml.m5.xlarge"
endpoint_instance_type = "ml.m5.xlarge"
    
role = sagemaker.get_execution_role()
print("Using IAM role arn: {}".format(role))

# only run from SageMaker notebook instance
if local_mode:
    !/bin/bash ./setup.sh

## Define the attributes of the training job
Use `job_name_prefix` to identify the sagemaker training job for this.

In [ ]:
job_name_prefix = 'Battlesnake-job'

## Define the metrics to evaluate your training job
The regex for this metric was defined based on what is printed in the training script `examples/train.py`

In [ ]:
metric_definitions = [
    {'Name': 'timesteps', 'Regex': '.*Mean timesteps ([-+]?[0-9]*[.]?[0-9]+([eE][-+]?[0-9]+)?)'},
]

## Define the hyperparameters of your job

In [ ]:
map_size_string = "[{}, {}]".format(map_size[0], map_size[1])
static_hyperparameters = {
    'qnetwork_type': "attention",
    'seed': 666,
    'number_of_snakes': 4,
    'episodes': 2200,
    'print_score_steps': 10,
    'activation_type': "softrelu",
    'state_type': 'one_versus_all',
    'sequence_length': 2,
    'repeat_size': 3,
    'kernel_size': 3,
    'starting_channels': 6,
    'map_size': map_size_string,
    'snake_representation': 'bordered-51s',
    'save_model_every': 700,
    'eps_start': 0.99,
    'models_to_save': 'local'
}

# Train your model here
Defines the estimator. Firstly, try run this with `local_model = True` to test your entry_point script

In [ ]:
estimator = MXNet(entry_point="train.py",
                  source_dir='battlesnake_src',
                  dependencies=["battlesnake_gym/"],
                  role=role,
                  train_instance_type=train_instance_type,
                  train_instance_count=1,
                  output_path=s3_output_path,
                  framework_version="1.6.0",
                  py_version='py3',
                  base_job_name=job_name_prefix,
                  metric_definitions=metric_definitions,
                  hyperparameters=static_hyperparameters
                 )
if local_mode:
    estimator.fit()

# Running hyperparameter optimisation
Start the Hyperparameter optimisation sagemaker jobs!

In [ ]:
hyperparameter_ranges = {
    'buffer_size': IntegerParameter(1000, 6000),
    'update_every': IntegerParameter(10, 20),
    'batch_size': IntegerParameter(16, 256),

    'lr_start': ContinuousParameter(1e-5, 1e-3),
    'lr_factor': ContinuousParameter(0.5, 1.0),
    'lr_step': IntegerParameter(5000, 30000),
    
    'tau': ContinuousParameter(1e-4, 1e-3),
    'gamma': ContinuousParameter(0.85, 0.99),
    
    'depth': IntegerParameter(10, 256),
    'depthS': IntegerParameter(10, 256),
}

In [ ]:
max_jobs = 3
max_parallel_jobs = 3

tuner = HyperparameterTuner(estimator,
                            objective_metric_name='timesteps',
                            objective_type='Maximize',
                            hyperparameter_ranges=hyperparameter_ranges,
                            metric_definitions=metric_definitions,
                            max_jobs=max_jobs,
                            max_parallel_jobs=max_parallel_jobs,
                            base_tuning_job_name=job_name_prefix)
if run_hpo:
    tuner.fit()

Now wait for the hyper parameter turner to complete

# Collect the target model
Obtain an s3 URL of the best model

In [ ]:
if run_hpo:
    best_training_job = tuner.best_training_job()
    best_model_path = "{}/{}/output/model.tar.gz".format(s3_output_path, best_training_job)
else:
    best_model_path = estimator.model_data
print("Best model location {}".format(best_model_path))

In [ ]:
s3_client = boto3.client('s3')
key = "/".join(best_model_path[5:].split("/")[1:])
copy_source = {'Bucket': s3_bucket, 'Key': key}
target_key = "battlesnake-pretrainedmodels/Model-{}x{}/Model.tar.gz".format(map_size[0], map_size[1])
s3_client.copy_object(Bucket=s3_bucket, CopySource=copy_source, Key=target_key)

# Update endpoint with your new model

In [ ]:
model_data = "s3://{}/{}".format(s3_bucket, target_key)

from sagemaker.mxnet import MXNetModel
mxnet_model = MXNetModel(model_data=model_data,
                             entry_point='predict.py',
                             role=role,
                             framework_version='1.6.0',
                             source_dir='battlesnake_inference',
                             name="battlesnake-mxnet",
                             py_version='py3')
predictor = mxnet_model.deploy(initial_instance_count=1,
                               instance_type=endpoint_instance_type,
                               update_endpoint=True,
                               endpoint_name='battlesnake-endpoint')

In [ ]:
data1 = np.zeros(shape=(1, 2, 3, map_size[0]+2, map_size[1]+2))
data2 = np.zeros(shape=(1, 2))
data3 = np.zeros(shape=(1, 2))
data4 = np.zeros(shape=(1, 2))
action = predictor.predict({"state": data1, "snake_id": data2, "turn_count": data3, "health": data4})
print("Predicted action is {}".format(action))

# Multi data model - WIP

In [ ]:
from sagemaker.multidatamodel import MultiDataModel
from sagemaker.mxnet import MXNetModel
mxnet_model = MXNetModel(model_data=best_model_path,
                         entry_point='predict.py',
                         role=role,
                         framework_version='1.6.0',
                         source_dir='battlesnake_inference',
                         name="battlesnake-mxnet",
                         py_version='py3', 
                         sagemaker_session=sage_session)
predictor = mxnet_model.deploy(initial_instance_count=1,
                               instance_type=endpoint_instance_type,
                               endpoint_name='battlesnake-endpoint')

In [ ]:
multi_model_path = "{}/battlesnake-pretrainedmodels".format(s3_output_path)

sagemaker_multidata_model = MultiDataModel("battlesnake-mdm", multi_model_path,
                                           model=sagemaker_model, sagemaker_session=sage_session)
predictor = sagemaker_multidata_model.deploy(initial_instance_count=1,
                                   instance_type=endpoint_instance_type,
                                   endpoint_name='battlesnake-endpoint-mdm')

In [ ]:
data1 = np.zeros(shape=(1, 2, 3, 17, 17))
data2 = np.zeros(shape=(1, 2))
data3 = np.zeros(shape=(1, 2))
data4 = np.zeros(shape=(1, 2))
predictor.predict({"state": data1, "snake_id": data2, "turn_count": data3, "health": data4}, target_model="Model-15x15.tar.gz")